In [ ]:
# =====================================================
# FAKE NEWS DETECTION – MINILM (Fixed Model ID)
# =====================================================

# 1. CÀI ĐẶT & IMPORT
!pip install -q transformers datasets torch scikit-learn pandas numpy psutil accelerate

import os, re, shutil, psutil, warnings
import pandas as pd
import numpy as np
import torch
from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.utils.class_weight import compute_class_weight
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    Trainer, TrainingArguments, EarlyStoppingCallback,
    DataCollatorWithPadding
)
from google.colab import drive

warnings.filterwarnings("ignore")

# 2. KIỂM TRA GPU
if torch.cuda.is_available():
    device_name = torch.cuda.get_device_name(0)
    print(f"✅ Device: {device_name} | VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ CẢNH BÁO: Bạn đang chạy trên CPU! Hãy chuyển Runtime sang T4 GPU để train nhanh hơn.")

# 3. MOUNT DRIVE
drive.mount('/content/drive', force_remount=True)
OUTPUT_DIR = "/content/drive/MyDrive/FakeNewsNet_MiniLM_Fixed"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 4. TẢI DATASET (FakeNewsNet)
print("\n⏳ Đang tải dataset FakeNewsNet...")
try:
    ds_gossip = load_dataset("rickstello/FakeNewsNet", "gossipcop", split="train")
    ds_politi = load_dataset("rickstello/FakeNewsNet", "politifact", split="train")
    dataset_full = concatenate_datasets([ds_gossip, ds_politi])
    df = pd.DataFrame(dataset_full)
except Exception as e:
    print(f"⚠️ Tải config con thất bại ({e}), tải bản default...")
    dataset = load_dataset("rickstello/FakeNewsNet", split="train")
    df = pd.DataFrame(dataset)

print(f"Tổng số mẫu: {len(df)}")

# 5. XỬ LÝ DỮ LIỆU
# Auto-detect columns
text_col = next((c for c in ['news_content', 'text', 'content', 'body'] if c in df.columns), None)
title_col = next((c for c in ['title', 'news_title', 'headline'] if c in df.columns), None)
label_col = next((c for c in ['real', 'label', 'class', 'fake'] if c in df.columns), None)

if not label_col: raise ValueError("❌ Không tìm thấy cột nhãn!")

df['label'] = df[label_col]
title_data = df[title_col].fillna('') if title_col else ""
text_data = df[text_col].fillna('') if text_col else ""
df['content'] = title_data + " [SEP] " + text_data

def clean_text(s):
    if not isinstance(s, str): return ""
    s = s.lower()
    s = re.sub(r'https?://\S+', ' ', s)
    s = re.sub(r'<.*?>', ' ', s)
    s = re.sub(r'[^a-z0-9\s]', ' ', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

print("🧹 Đang làm sạch...", end="")
df['content'] = df['content'].apply(clean_text)
df = df[df['content'].str.len() > 20].drop_duplicates(subset=['content'])
print(f" → Sau xử lý: {len(df):,}")

# Class Weights
classes = np.unique(df['label'])
class_weights = compute_class_weight('balanced', classes=classes, y=df['label'])
class_weight_dict = {k: float(v) for k, v in zip(classes, class_weights)}

# Split
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])

dataset_dict = DatasetDict({
    "train": Dataset.from_pandas(train_df[['content','label']].reset_index(drop=True)),
    "validation": Dataset.from_pandas(val_df[['content','label']].reset_index(drop=True)),
    "test": Dataset.from_pandas(test_df[['content','label']].reset_index(drop=True))
})

# 6. MODEL & TOKENIZER (ĐÃ SỬA ID)
# Sử dụng 'nreimers/MiniLM-L6-H384-uncased' thay vì microsoft/...
MODEL_NAME = "nreimers/MiniLM-L6-H384-uncased"
print(f"\n⚙️ Đang tải model: {MODEL_NAME}")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

# Config nhãn (FakeNewsNet: 0=Fake, 1=Real hoặc ngược lại tùy cột 'real')
model.config.id2label = {0: "Fake", 1: "Real"}
model.config.label2id = {"Fake": 0, "Real": 1}

def tokenize_fn(batch):
    return tokenizer(batch["content"], truncation=True, max_length=384, padding=False)

tokenized = dataset_dict.map(tokenize_fn, batched=True, batch_size=1000, remove_columns=['content'])
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 7. TRAINER SETUP
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=5,               
    per_device_train_batch_size=16,    
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=1,
    learning_rate=2e-5,              
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=torch.cuda.is_available(),
    report_to="none"
)

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        w = torch.tensor(list(class_weight_dict.values()), dtype=torch.float32, device=model.device)
        loss_fct = torch.nn.CrossEntropyLoss(weight=w)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

# 8. TRAIN
print("\n🚀 BẮT ĐẦU HUẤN LUYỆN...")
trainer.train()

print("\n🎯 KẾT QUẢ TRÊN TEST SET:")
print(trainer.evaluate(tokenized["test"]))

# Save final
final_path = os.path.join(OUTPUT_DIR, "final_minilm_fixed")
trainer.save_model(final_path)
tokenizer.save_pretrained(final_path)
print(f"\n✅ Đã lưu model tại: {final_path}")

✅ Device: Tesla T4 | VRAM: 15.8 GB
Mounted at /content/drive

⏳ Đang tải dataset FakeNewsNet...
⚠️ Tải config con thất bại (BuilderConfig 'gossipcop' not found. Available: ['default']), tải bản default...
Tổng số mẫu: 23196
🧹 Đang làm sạch... → Sau xử lý: 21,287

⚙️ Đang tải model: nreimers/MiniLM-L6-H384-uncased


tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]


🚀 BẮT ĐẦU HUẤN LUYỆN...


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.504700,0.487120,0.775012,0.787038,0.817710,0.775012
2,0.436000,0.499554,0.793800,0.803890,0.828802,0.793800
3,0.390000,0.482590,0.799906,0.809063,0.830641,0.799906
4,0.353100,0.491126,0.827149,0.830992,0.837145,0.827149
5,0.323300,0.499425,0.826209,0.830600,0.838059,0.826209



🎯 KẾT QUẢ TRÊN TEST SET:


{'eval_loss': 0.4982742667198181, 'eval_accuracy': 0.8149365899483325, 'eval_f1': 0.8210820936606682, 'eval_precision': 0.8328827248720004, 'eval_recall': 0.8149365899483325, 'eval_runtime': 0.7173, 'eval_samples_per_second': 2967.927, 'eval_steps_per_second': 93.401, 'epoch': 5.0}

✅ Đã lưu model tại: /content/drive/MyDrive/FakeNewsNet_MiniLM_Fixed/final_minilm_fixed
